#Reddit Fake Post Detection(by Looking Only at the Title)

##Problem Formulation
False information on the Internet has caused many social problems due to the raise of social network and its role in different domains such as politics. In this project, we are going to predict if a specific reddit post is fake news or not, by looking at its title.

## Data Loading and Exploration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import libiraries
import pandas as pd
import numpy as np

#read train and test data
data=pd.read_csv('/content/drive/MyDrive/xy_train[1].csv',index_col='id')
test_=pd.read_csv('/content/drive/MyDrive/x_test[1].csv')

In [ ]:
#display train data
data

,text,label
id,,
265723,A group of friends began to volunteer at a hom...,0
284269,British Prime Minister @Theresa_May on Nerve A...,0
207715,"In 1961, Goodyear released a kit that allows P...",0
551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...
70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
189377,Nigerian Prince Scam took $110K from Kansas ma...,1
93486,Is It Safe To Smoke Marijuana During Pregnancy...,0


In [ ]:
#display test data
test_

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from Y...
3,3,"As Trump Accuses Iran, He Has One Problem: His..."
4,4,"""Believers"" - Hezbollah 2011"
...,...,...
59146,59146,Bicycle taxi drivers of New Delhi
59147,59147,Trump blows up GOP's formula for winning House...
59148,59148,Napoleon returns from his exile on the island ...
59149,59149,Deep down he always wanted to be a ballet dancer


In [ ]:
#display the no. of nulls in each columns
data.isnull().sum().sort_values(ascending=False)

text     0
label    0
dtype: int64

In [ ]:
#display data information
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 265723 to 34509
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    60000 non-null  object
 1   label   60000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [ ]:
#check if there is duplicate in data
data.duplicated().sum()

345

##Data Cleaning and preprocessing


In [ ]:
# Drop duplicate rows
data.drop_duplicates(subset='text', inplace=True)

In [ ]:
#replace label 2 with label 1
data["new_label"] = 0
data.loc[data["label"] == 0, "new_label"] = 0
data.loc[data["label"] > 0, "new_label"] = 1

#checking for the distribution of grade_bad
data['new_label'].value_counts()

0    31949
1    27696
Name: new_label, dtype: int64

In [ ]:
#display new data shape
data.shape

(59645, 3)

In [ ]:
from bokeh.io import output_notebook
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import pickle
import sklearn
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()
from pathlib import Path
from textblob import TextBlob
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

#use lemmatizer to convert the word to root word
#use stop words to remove words that not add much value to the meaning of text
lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words("english"))


def clean_text(text, for_embedding=False):

    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
    #use textblob to correct spelling mistakes
    #m = TextBlob(text)
    #text=m.correct()

    #divide rows into sentences(tokens) to be easier to understod
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no lemmatizing, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            lemmatizer.lemmatize(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data.text.describe()

count                                                 59645
unique                                                59645
top       A group of friends began to volunteer at a hom...
freq                                                      1
Name: text, dtype: object

In [ ]:
#check if train data balanced or not
data["new_label"].value_counts(normalize=True)

0    0.535653
1    0.464347
Name: new_label, dtype: float64

In [ ]:
#split columns
test=test_['text']
x=data['text']
y=data['new_label']
x,y,test

(id
 265723    A group of friends began to volunteer at a hom...
 284269    British Prime Minister @Theresa_May on Nerve A...
 207715    In 1961, Goodyear released a kit that allows P...
 551106    Happy Birthday, Bob Barker! The Price Is Right...
 8584      Obama to Nation: 聙"Innocent Cops and Unarmed Y...
                                 ...                        
 70046     Finish Sniper Simo H盲yh盲 during the invasion o...
 189377    Nigerian Prince Scam took $110K from Kansas ma...
 93486     Is It Safe To Smoke Marijuana During Pregnancy...
 140950    Julius Caesar upon realizing that everyone in ...
 34509     Jeff Bridges Releasing 鈥楽leeping Tapes,鈥?a New...
 Name: text, Length: 59645, dtype: object, id
 265723    0
 284269    0
 207715    0
 551106    0
 8584      0
          ..
 70046     0
 189377    1
 93486     0
 140950    0
 34509     1
 Name: new_label, Length: 59645, dtype: int64, 0                                               stargazer 
 1                            

Data banaced and we don't need data augmentation

##Models Building

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit

###Trial 1
Use pipeline with GridSearch, validation set and logistic regression model with word-level vectorizer.

In [ ]:
# Further split the original training set to a train and a validation set
X_train, X_val, y_train, y_val = train_test_split(x, y, train_size = 0.8, stratify = y, random_state = 2022)
# Create a list where train data indices are -1 and validation data indices are 0
split_index = [-1 if i in X_train.index else 0 for i in x.index]
# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

#set vectorizer hyperparameters
pipe = Pipeline([('cvec', TfidfVectorizer(preprocessor=clean_text,analyzer="word", max_df=0.3, min_df=10, norm="l2")),    
                 ('lr', LogisticRegression(solver='sag'))])
# Tune GridSearchCV
pipe_params = {'cvec__ngram_range': [(1,1), (2,2), (1,3)],'lr__C': [0.01, 0.1,1], 'lr__penalty': ['l1', 'l2']}
gs = GridSearchCV(pipe, param_grid=pipe_params,  scoring="roc_auc", cv=pds)
#train model with gridsearchcv
gs.fit(x, y);
#predict best score and params
print("Train score", gs.score(x, y))
print("Best params:", gs.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
9 fits failed out of a total of 18.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File 

Train score 0.9201564839608707
Best params: {'cvec__ngram_range': (1, 3), 'lr__C': 1, 'lr__penalty': 'l2'}


Score is 0.83705 on kaggle

I think it will stay the best score in machine learning model but in NN models the score will be better.

In [ ]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_['id']
submission['label']=gs.predict_proba(test)[:,1]
submission.to_csv('sample_submission_walkthrough1.csv', index=False)

###Trial 2
Use MultinominalNB with word analyzer and Gridsearch

I think it will be good

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
#set vectorizer hyperparameters
pipe = Pipeline([('cvec', TfidfVectorizer(preprocessor=clean_text,analyzer="word", max_df=0.3, min_df=10, norm="l2")),    
                 ('nb', MultinomialNB())])
# Tune GridSearchCV
pipe_params = {'cvec__ngram_range': [(1,1), (2,2), (1,3)],
               'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
gs = GridSearchCV(estimator=pipe, param_grid=pipe_params,  scoring="roc_auc", cv=pds)
#train model with gridsearchcv
gs.fit(x, y);
#predict best score and params
print("Train score", gs.score(x, y))
print("Best params:", gs.best_params_)

Train score 0.8961851993624514
Best params: {'cvec__ngram_range': (1, 3), 'nb__alpha': 1}


Score on kaggle is: 0.82452

Multinomial is good too.

In [ ]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_['id']
submission['label']=gs.predict_proba(test)[:,1]
submission.to_csv('sample_submission_walkthrough2.csv', index=False)

###Trial 3
Model with randomForest, character-level vectorizer and GridSearch with validation set

I think RandomForest will overrfitt data and will not be good in test.

In [ ]:
# Randomforest pipeline setup
rf_pipe = Pipeline([('tvec',  TfidfVectorizer(preprocessor=clean_text,analyzer="char", max_df=0.3, min_df=10, norm="l2")),
                    ('rf', RandomForestClassifier())])
# Setting up randomforest params
rf_params = {'tvec__max_features':[2000],'tvec__ngram_range': [(1, 2)],'tvec__stop_words': ['english'],
             'rf__max_depth': [1000],'rf__min_samples_split': [100],'rf__max_leaf_nodes': [None]}
             
rf_pipe.fit(x, y)
# Setting up GridSearch for TFIDFVectorizer
rf_gs = GridSearchCV(rf_pipe, param_grid=rf_params, scoring="roc_auc", cv = pds, verbose = 1, n_jobs = -1)
# Fitting Randomforest CV GS
rf_gs.fit(x, y)
#predict best score
print("Train score", rf_gs.score(x, y))
print("Best params:", rf_gs.best_params_)

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Train score 0.9797152498008315
Best params: {'rf__max_depth': 1000, 'rf__max_leaf_nodes': None, 'rf__min_samples_split': 100, 'tvec__max_features': 2000, 'tvec__ngram_range': (1, 2), 'tvec__stop_words': 'english'}


Score on kaggle is: 0.75818

As I expected train score is 0.98 and test on kaggle is 0.72

RandomForest always like to overfitt

In [ ]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_['id']
submission['label']=rf_gs.predict_proba(test)[:,1]
submission.to_csv('sample_submission_walkthrough3.csv', index=False)

###Trial 4
XGBoost Model with bayesian search and word-level vectorizer with validation set

It maybe better than randomforest because overfitting problems have been solved here.

In [ ]:
# combine the preprocessor with the model as a full tunable pipeline
# we gave them a name so we can set their hyperparameters
full_pipline = Pipeline(steps=[('cvec', TfidfVectorizer(preprocessor=clean_text,analyzer="word", max_df=0.3, min_df=10, norm="l2")), 
                                 ('my_classifier', XGBClassifier(n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1))])
full_pipline.fit(x,y)
param_grid = {'cvec__ngram_range': [(1,1), (2,2), (1,3)],
             'my_classifier__max_depth':[5,10, 20, 30,40,50] }

grid_xg = GridSearchCV(full_pipline, param_grid=param_grid, scoring="roc_auc", cv = pds, verbose = 1, n_jobs = -1)
grid_xg.fit(x, y)
#predict best score
print('best score {}'.format(grid_xg.best_score_))
print('best score {}'.format(grid_xg.best_params_))

Fitting 1 folds for each of 18 candidates, totalling 18 fits
best score 0.8571246540041436
best score {'cvec__ngram_range': (1, 3), 'my_classifier__max_depth': 50}
CPU times: user 8min 17s, sys: 7.71 s, total: 8min 25s
Wall time: 28min 57s


Score on kaggle is: 0.82029

In [ ]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_['id']
submission['label']=grid_xg.predict_proba(test)[:,1]
submission.to_csv('sample_submission_walkthrough4.csv', index=False)

###Trial 5
MLP classifier with word level vectorizer and gridsearchcv

I expect in MLP the model will perform better and score will be the best.

In [ ]:
full_pipline_5 = Pipeline(steps=[('cvec', TfidfVectorizer(preprocessor=clean_text,analyzer="word", max_df=0.3, min_df=10, norm="l2")), 
                                 ('my_classifier',MLPClassifier(random_state=1,solver="adam",
                                  hidden_layer_sizes=(224, 120, 12,),activation="relu",n_iter_no_change=10))])

param_grid_5 = {'cvec__ngram_range': [(1,1), (2,2), (1,3)]}
grid_mlp = GridSearchCV(full_pipline_5,param_grid_5,scoring="roc_auc", cv = pds, verbose = 1, n_jobs = -1)
grid_mlp.fit(x, y)
#predict best score
print('best score {}'.format(grid_mlp.best_score_))
print('best score {}'.format(grid_mlp.best_params_))

Fitting 1 folds for each of 3 candidates, totalling 3 fits
best score 0.8509244449869061
best score {'cvec__ngram_range': (1, 1)}
CPU times: user 11min 54s, sys: 9min 12s, total: 21min 7s
Wall time: 27min 18s


Score on kaggle: 0.80617

Good score but not as much as I expect, I thought MLP will give us best score.

In [ ]:
#predict output and save submission
submission = pd.DataFrame()
submission['id'] = test_['id']
submission['label']=grid_mlp.predict_proba(test)[:,1]
submission.to_csv('sample_submission_walkthrough5.csv', index=False)

##Problem Formulation

###The Problem is:

We have data contains text column and each row include title and each title will be classed as fake or not.
the problem is that the data contains contains various forms of words so we should apply text preprocessing techniques ti clean it. 

###What is the input?

Text coulmn includes titles that will be classified as fake or not.

###What is the output?

Probability of how much is the title classified as fake.

###What data mining function is required?


*   Drop duplicate
*   Drop useless rows
*   re.sub
*   re.compile
*   re.IGNORECASE
*   word.lower()
*   value_counts

*   Steeming
*   Stop words










###What could be the challenges?

To remove operation signs from texts and stop words to make text more clear.
To make computer understand human words, In a normal conversation between humans, things are often unsaid, whether in the form of some signal, expression, or just silence.
Nevertheless, we, as humans, have the capacity to understand the
underlying intent of the conversation, which a computer lacks.
A second difficulty is owing to ambiguity in sentences. This may be at
the word level, at the sentence level, or at the meaning level.

###What is the impact?

The model will truly understand human language and classify if news is fake or not from it's title only.

###What is an ideal solution?

Logistic regression model with word-level victorizer and GridSearchCV.

###What is the experimental protocol used and how was it carried out?

tf-idf Char level vectorizer and tf-idf Word level vectorizer.
Both are good and each one of them is good with sort of data.
We cannot decide which is better than other, It depends only on trained data.

###What preprocessing steps are used?

*   We dropped duplicate rows and clear data that includes false label.
*   We cleared stop words.
*   We cleared operations signs to make classifying easier.
*   Convert upper cases to lower.
*   Use tf-idf with word vectorizer and char vectorizer.










##Questions


###🌈 What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?

Character n-gram compute how much charachter repeated depends on the selected number, but word n-gram compute how much word repeated depends on selected number.

Character Tokenizers handles OOV words coherently by preserving the information of the word. It breaks down the OOV word into characters and represents the word in terms of these characters. It also limits the size of the vocabulary. since the 26 vocabulary contains a unique set of characters.

###🌈 What is the difference between stop word removal and stemming? Are these techniques language-dependent?

Stop word removes words that not add much value to the meaning of the text.

like (“the”, “is”, “in”, “for”, “where”, “when”, “to”, “at” etc.).

Steeming is a text normalization technique that cuts off the end or beginning of a word by taking into account a list of common prefixes or suffixes that could be found in that word.

It is a rudimentary rule-based process of stripping the suffixes (“ing”, “ly”, “es”, “s” etc) from a word.

It is highly dependent on the task we are performing.

###🌈 Is tokenization techniques language dependent? Why?


Yes, Tokenization is breaking the raw text into small chunks. Tokenization breaks the raw text into words, sentences called tokens. These tokens help in understanding the context or developing the model for the NLP. The tokenization helps in interpreting the meaning of the text by analyzing the sequence of the words.

###🌈 What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?

Count Vectorizer is a way to convert a given set of strings into a frequency representation.

TF-IDF stands for Term Frequency — Inverse Document Frequency and is a statistic that aims to better define how important a word is for a document, while also taking into account the relation to other documents from the same corpus.





https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/

https://towardsdatascience.com/tokenization-for-natural-language-processing-a179a891bad4

https://towardsdatascience.com/text-pre-processing-stop-words-removal-using-different-libraries-f20bac19929a

https://www.analyticsvidhya.com/blog/2019/08/how-to-remove-stopwords-text-normalization-nltk-spacy-gensim-python/